In [172]:
from workalendar.europe import *
import requests
from datetime import datetime, timedelta
import json
import itertools
from typing import List
from typing_extensions import TypedDict, TypeAlias

In [164]:
calendarific_api_key = "2a7fb01b189f3d6735a2ab8d797b7a12dcba03d9"

def get_public_holidays_by_country(country_code):
    current_year = datetime.now().year
    current_year_str = [current_year - 1, current_year, current_year + 1]
    public_holidays = []

    try:
        for year in current_year_str:
            url = f"https://calendarific.com/api/v2/holidays?&api_key={calendarific_api_key}&country={country_code}&year={year}&type=national"
            res = requests.get(url)
            if (res.status_code == 200):
                public_holidays += [{'date': holiday['date']['datetime'], 'country': holiday['country']['id']} for holiday in res.json()['response']['holidays']]
            else:
                raise Exception(res.reason)
    except Exception as error:
        print(error)

    return public_holidays

In [173]:
class DateDict(TypedDict, total=True):
  year: int
  month: int
  day: int

class ApiObject(TypedDict):
  date: DateDict
  country: str

ApiObjectCollection: TypeAlias = List[ApiObject]

In [174]:
def is_business_day(date: datetime, holidays: ApiObjectCollection) -> bool: 
    if date.weekday() >= 5:
        return False
    for holiday in holidays:
        if date.date() == datetime(**holiday['date']).date():
            return False
    return True

def find_business_date(date: datetime, public_holidays: ApiObjectCollection, days_before: int) -> datetime:
    count = 0
    while count < days_before:
        date -= timedelta(days=1)
        if is_business_day(date, public_holidays):
            count += 1

    return date

def find_common_business_date(event_cut_off_date: datetime, global_holidays: ApiObjectCollection):
    while True:
        if (is_business_day(event_cut_off_date, global_holidays)):
            return event_cut_off_date
        event_cut_off_date -= timedelta(days=1)

def find_valid_cut_off_date(cut_off_date: datetime, days_before: int, fund_origin_country: str, countries_involved: List[str] ):
    country_holidays_map = {}
    all_countries = [fund_origin_country] + countries_involved

    for country_code in all_countries:
        country_holidays_map[country_code] = get_public_holidays_by_country(country_code)

    countries_involved_holidays = list(itertools.chain(*[country_holidays_map[it] for it in all_countries]))

    fund_cut_off_date = find_business_date(cut_off_date, country_holidays_map[fund_origin_country], days_before)
    fund_common_cut_off_date = find_common_business_date(fund_cut_off_date, countries_involved_holidays)
    return fund_common_cut_off_date

In [178]:
test_date = datetime(2023, 11, 13)
date = find_valid_cut_off_date(
    cut_off_date=test_date,
    days_before=1,
    fund_origin_country='HK',
    countries_involved=['US']
)
print(date)

2023-11-09 00:00:00
